<center><img src="images/header.png"></center>

<h1><center>Рогожин Денис</center></h1>
<hr>
<h1><center>Домашняя работа</center></h1>

Данное ДЗ является довольно творческим и оно требует довольно много самостоятельной работы, так что не стоит с ним мешкать =)

Вам предлагается выбрать сообщество (или сообщества) в сети ВКонтакте и проанализировать его (их) контингент. 


**Правила:**
* Отчеты принимаются до **23:59 19 ноября**. Вы должны залить ваш ноутбук в github дать на него ссылку в следующей [форме](https://forms.gle/88d9CfaJN5gPV4bB8).
* Работоспособность будет проверяться через [colab](https://colab.research.google.com) (можете сразу давать ссылку на него). Если ноутбук не будет исполняться (падать по ошибке) то все, что будет написано после ошибки не будет учитываться при формировании оценки. Для самопроверки, выполните аналог restart kernel and run all cells в colab
* Вы должны включить код, с помощью которого вы общались с API вконтакте, но для экономии времени, заранее предусмотреть возможность загрузки уже собранных данных (например, как это было реализовано в ноутбуках по кластеризации через `wget` и dropbox или raw данными в github)

**Пожелания:**
* Выбирайте для анализа небольшие сообщества (до 5к), желательно чтобы вы сами в них состояли - это ускорит вычисления и упростит интерпретацию
* Отнеситесь к этой работе как к исследованию - больше графиков и комментариев!
* Красивые визуализации лучше делать через gephi!

# Загрузка данных (2 балла)

Изучите следующие материалы
* [Знакомство с API](https://vk.com/dev/first_guide)
* [Туториал](https://nbviewer.jupyter.org/github/allatambov/Py-programming-3/blob/master/15-06/lect-vk-api.ipynb) по выгрузке данных с помощью VKAPI
* Про метод execute - [туть](https://vk.com/dev/execute) и [туть](http://www.cyberforum.ru/python-web/thread1730446.html)

Выберите сообщество и загрузите
* Идентификаторы всех [участников сообщества](https://vk.com/dev/groups.getMembers)
* [Списки друзей](https://vk.com/dev/friends.get) участников и их признаки (пол, возраст, университет и все, что посчитаете нужным - нам потом группировать по этим атрибутам)
* [Пересечение](https://vk.com/dev/friends.getMutual) между списками друзей

Таким образом вы должны получить замкнутую группу аккаунтов с их друзьями и связями между ними.

Далее - соберите их этого граф в networkX и сохраните в graphml или gml формате

**Замечания:**
* Вконтакте может ограничить вам скорость запросов, поэтому иногда имеет смысл ставить небольшую паузу (`time.sleep(1)`) между запросами и сохранять предварительные результаты
* Код выгрузки может быть написан как с помощью модуля `vk` так и просто с помощью requests
* Некоторые пользователи могли сделать свой аккаунт приватным, что может вызвать ошибку в методе `getMutual`. Чтобы этого избежать придется проверять каждого на приватность с помощью `users.get` и аттрибута `is_closed`


In [12]:
"""
import vk
import time
token='ТУТ БЫЛ ТОКЕН, НО ЕГО НЕЛЬЗЯ НИКОМУ ПЕРЕДАВАТЬ, ПОЭТОМУ Я ЕГО СТЕР'
session = vk.Session(access_token = token)
api = vk.API(session)

# Импортирую только пол, так как остальные признаки отсутствуют у многих
members=api.groups.getMembers(group_id=198317113,fields=['sex','universities','city','bdate'],v = 5.89) 
#Выбрал группу своей кафедры 
member_id1=members['items']

#оставляю только тех, у кого открыта страница в вк
member_id=[]
for m in member_id1:
    try:
        if m['is_closed']==False:
            member_id.append(m)
    except: 
        print('no fieild is_closed')
#у некоторых людей почему-то нет этого поля(может быть из-за удаления/блокировки страницы, поэтому ловлю исключения

#получаю список, каждый элемент которого список друзей участников
member_friends=[]
for m in member_id:
        member_friends.append(api.friends.get(user_id=m['id'],fields=['sex','universities','city','bdate'],v = 5.89))
        time.sleep(1)
        
friends_list=[]
for m in member_friends:
    friends_list.append(m['items'])
    

#остается получить списки друзей
#метод Get mutual мне в этом не помог:с 
#очень быстро исчерпывается лимит обращений: VkAPIError: 29. Rate limit reached. 
#но я выкрутился и сделал так: выгрузил списки друзей для всех друзей участников и пересек их со списками друзей для участников.
 
    
#составляю список всех друзей участников, у кого открыта страница    

friend_id=[]
def find_in_friend_id(x):
    for m in friend_id:
        if x==m['id']:
            return True
    return False       
 
for i in range (0,len(friends_list)):
    for m in friends_list[i]:
        try:
            if m['is_closed']==False and not find_in_friend_id(m['id']):
                friend_id.append(m)
        except: 
            print('no fieild is_closed')
           
 #строю список друзей друзей участников        
member_friends_friends=[]
for m in friend_id:
        member_friends_friends.append(api.friends.get(user_id=m['id'],fields=['sex','universities','city','bdate'],v = 5.89))
        time.sleep(0.4)
        
member_friends_friends_t=[]
for m in member_friends_friends:
    member_friends_friends_t.append(m['items'])
   
 #пересечение списка друзей между i-ым и j-ым юзером
def find_friends(fr_i,fr_j):
    res=[]
    for i in range (0,len(fr_i)):
        for j in range (0,len(fr_j)):
            if fr_i[i]['id']==fr_j[j]['id']:
                x=fr_i[i]['id']
                res.append(x)
                break
    return res


#строю список общих друзей: каждый его элемент - это список вида [i j [k1,...,kn]]
#i,j - id двух пользователей, [k1,...,kn]- список их общих друзей
common_friends=[]
for i in range(0,len(member_friends_friends_t)):
    for j in range(0,len(member_id)):
        x=find_friends(member_friends_friends_t[i],friends_list[j])
        y=[friend_id[i]['id'],member_id[j]['id'],x]
        common_friends.append(y)    
    
"""    

"\nimport vk\nimport time\ntoken='ТУТ БЫЛ ТОКЕН, НО ЕГО НЕЛЬЗЯ НИКОМУ ПЕРЕДАВАТЬ, ПОЭТОМУ Я ЕГО СТЕР'\nsession = vk.Session(access_token = token)\napi = vk.API(session)\n\n# Импортирую только пол, так как остальные признаки отсутствуют у многих\nmembers=api.groups.getMembers(group_id=198317113,fields=['sex','universities','city','bdate'],v = 5.89) \n#Выбрал группу своей кафедры \nmember_id1=members['items']\n\n#оставляю только тех, у кого открыта страница в вк\nmember_id=[]\nfor m in member_id1:\n    try:\n        if m['is_closed']==False:\n            member_id.append(m)\n    except: \n        print('no fieild is_closed')\n#у некоторых людей почему-то нет этого поля(может быть из-за удаления/блокировки страницы, поэтому ловлю исключения\n\n#получаю список, каждый элемент которого список друзей участников\nmember_friends=[]\nfor m in member_id:\n        member_friends.append(api.friends.get(user_id=m['id'],fields=['sex','universities','city','bdate'],v = 5.89))\n        time.sleep(1)\n

In [13]:
"""
import networkx as nx

g = nx.Graph()
#добавляю участников сообщества в граф
g.add_nodes_from((m['id'], {'sex': m['sex'], 
                                'first_name': m['first_name'], 
                                'last_name': m['last_name'], 
                                 }) for m in member_id)

#добавляю друзей участников в граф
for i in range (0,len(friends_list)):
    g.add_nodes_from((m['id'], {'sex': m['sex'], 
                                'first_name': m['first_name'], 
                                'last_name': m['last_name'], 
                                 }) for m in friends_list[i])
    
#соединяю ребрами участников и их друзей    
for i in range (0,len(friends_list)):
    for m in friends_list[i]:
        g.add_edge(m['id'],member_id[i]['id'])  
        
        
#остается соединить дугами друзей разных участников между собой
#метод Get mutual мне в этом не помог:с 
#очень быстро исчерпывается лимит обращений: VkAPIError: 29. Rate limit reached. 
#но я выкрутился и сделал так: выгрузил списки друзей для всех друзей участников и пересек их со списками друзей для участников.

for m in common_friends:
    i=m[0]
    j=m[1]
    com_fr_i_j=m[2]
    for x in com_fr_i_j:
        g.add_edge(i,x)
        g.add_edge(j,x) 
        
nx.write_graphml(g, 'gr.graphml') 
"""

"\nimport networkx as nx\n\ng = nx.Graph()\n#добавляю участников сообщества в граф\ng.add_nodes_from((m['id'], {'sex': m['sex'], \n                                'first_name': m['first_name'], \n                                'last_name': m['last_name'], \n                                 }) for m in member_id)\n\n#добавляю друзей участников в граф\nfor i in range (0,len(friends_list)):\n    g.add_nodes_from((m['id'], {'sex': m['sex'], \n                                'first_name': m['first_name'], \n                                'last_name': m['last_name'], \n                                 }) for m in friends_list[i])\n    \n#соединяю ребрами участников и их друзей    \nfor i in range (0,len(friends_list)):\n    for m in friends_list[i]:\n        g.add_edge(m['id'],member_id[i]['id'])  \n        \n        \n#остается соединить дугами друзей разных участников между собой\n#метод Get mutual мне в этом не помог:с \n#очень быстро исчерпывается лимит обращений: VkAPIError: 29. Rate 

In [14]:
import networkx as nx
g = nx.read_graphml('gr.graphml')


# Визуализация (1 балл)

* Откройте полученный граф в gephi
* Выберите наиболее удачный на ваш взгляд алгоритм прорисовки
* Добавьте изменение размера вершины в зависимости от степени вершины
* Вставьте скрин-шот ниже

![](simple_graph.jpg)
Вот такой получился граф после примерно часа ожидания работы алгоритма:)))

# Центральности вершин (2 балла)

Возможно, перед тем как пойти дальше, стоит немного почитстить граф - с помощью [kcore](https://www.google.com/search?client=firefox-b-d&q=networkx+kcore)

* Найдите наиболее влиятельные вершины с точки зрения degree, closeness и betweenness centrality
* Выделите их на картинке в gephi

In [15]:

#btw = nx.centrality.betweenness_centrality(g)
#nx.set_node_attributes(g, btw, "betweenness")
#cl = nx.centrality.closeness_centrality(g)
#nx.set_node_attributes(g, cl, "closeness")
#de = nx.centrality.degree_centrality(g)
#nx.set_node_attributes(g, de, "degree")
#nx.write_graphml(g, 'final_graph.graphml')



# betweenness centrality
![](btw_graph.jpg)

# closeness 
![](cl_graph.jpg)

# degree
![](de_graph.jpg)

# Сообщества... в сообществе (2 балла)

В этом задании надо примерить 2 алгоритма выявления сообществ и сравнить их результаты

Выберите алгоритм выделения сообществ на графах, например
* label propagation или 
* алгоритм на основе edge betweeness

и алгоритм на основе похожести вершин, например 
* расчитатайте попарную похожесть по jaccard, косинусному расстоянию или корреляцию
* при необходимости переведите "похожесть" в "расстояние" (чем больше значение тем "дальше" объекты друг от друга)
* подайте матрицу попарных расстояний в алгоритм кластеризации, например в DBSCAN

Для обоих вариантов
* Выделите сообщества, опираясь на максимальное значение модулярности для полученного разбиения
* Изобразите граф с вершинами, раскрашенными в цвет своего сообщества в gephi
* Попробуйте проинтерпретировать полученные сообщества, кто эти люди, почему они связаны?
* Какой метод на ваш взгляд сработал лучше?

In [16]:
### Ваш код, картинки и комментарии ниже